In [5]:
import numpy as np
import pandas as pd
import pickle

df = pd.read_csv(r"C:\Users\shree\Desktop\Personal\Projects\Project HR\HR dataset Project.csv")

## Replacing missing values in educatuion with mode of age category
# Step 1: Create a function to fill null values in 'education' based on 'age'
def fill_education_by_age(row):
    age = row['age']

    # Get the mode of education for individuals with the same age
    mode_education = df[(df['age'] == age)]['education'].mode().values

    # If there's a mode value, use it; otherwise, keep the original value
    return mode_education[0] if len(mode_education) > 0 else row['education']

#Apply the function to fill null values in the 'education' column
df['education'] = df.apply(lambda row: fill_education_by_age(row) if pd.isnull(row['education']) else row['education'], axis=1)

## Replacing Null values in previous_year_rating  with 0 as lenght of service is 1 year for employees for them i.e they are new employees
df['previous_year_rating'].fillna(0,inplace=True)

## Log transform
df[['age', 'length_of_service']] = np.log(df[['age', 'length_of_service']])

from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
cols = df[['avg_training_score']]
scaled = scalar.fit_transform(cols)

##ordinal encoding education column
from sklearn.preprocessing import OrdinalEncoder


Education = ['Below Secondary',"Bachelor's","Master's & above"]

enc = OrdinalEncoder(categories=[Education])

df[['education']] = enc.fit_transform(df[['education']])

##onehotencoding on gender recruitmentchannel and Department
from numpy import int32
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False,dtype=int32,drop = 'first')
cols = df[['gender','recruitment_channel','department']]
onehot_encoded = ohe.fit_transform(cols)
# Get the feature names
feature_names = ohe.get_feature_names_out(['gender', 'recruitment_channel','department'])
# Create a new DataFrame with the encoded data and feature names
df_encoded = pd.DataFrame(onehot_encoded, columns=feature_names)
# Concatenate the original DataFrame and the encoded DataFrame
df = pd.concat([df, df_encoded], axis=1)
# Droping columns 'gender','recruitment_channel','department'
df.drop(['gender','recruitment_channel','department'],axis=1,inplace=True)

## Encoding region with the region number
### extract number from region name

df['region'] = df['region'].str.extract('(\d+)')

## checkinf data type of column
df['region'].dtype
df['region'] = df['region'].astype(int)

# Using standardScalar for Scaling
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
cols = df[['avg_training_score']]
scaled = scalar.fit_transform(cols)
## Droping unscaled columns
df.drop(['avg_training_score'],axis =1,inplace = True)
# Create a new DataFrame with the encoded data and feature names
scaled_col = pd.DataFrame(scaled, columns=['avg_training_score'])
# Concatenate the original DataFrame and the encoded DataFrame
df = pd.concat([df, scaled_col], axis=1)

df.drop('employee_id',axis = 1 ,inplace = True)

x = df.drop(['is_promoted'],axis = 1) ## independent variable
y = df['is_promoted']

from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.25,random_state= 0)

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=15).fit(x_train,y_train)

C:\Users\shree\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [3]:
model

RandomForestClassifier(n_estimators=15)

In [6]:
df.head()

,region,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,is_promoted,gender_m,...,recruitment_channel_sourcing,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,avg_training_score
0,7,2.0,1,3.555348,5.0,2.079442,1,0,0,0,...,1,0,0,0,0,0,0,1,0,-1.075931
1,22,1.0,1,3.401197,5.0,1.386294,0,0,0,1,...,0,0,0,0,1,0,0,0,0,-0.253282
2,19,1.0,1,3.526361,3.0,1.945910,0,0,0,1,...,1,0,0,0,0,0,0,1,0,-1.001145
3,23,1.0,2,3.663562,1.0,2.302585,0,0,0,1,...,0,0,0,0,0,0,0,1,0,-1.001145
4,26,1.0,1,3.806662,3.0,0.693147,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0.718939


In [7]:
x

,region,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,gender_m,recruitment_channel_referred,recruitment_channel_sourcing,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,avg_training_score
0,7,2.0,1,3.555348,5.0,2.079442,1,0,0,0,1,0,0,0,0,0,0,1,0,-1.075931
1,22,1.0,1,3.401197,5.0,1.386294,0,0,1,0,0,0,0,0,1,0,0,0,0,-0.253282
2,19,1.0,1,3.526361,3.0,1.945910,0,0,1,0,1,0,0,0,0,0,0,1,0,-1.001145
3,23,1.0,2,3.663562,1.0,2.302585,0,0,1,0,0,0,0,0,0,0,0,1,0,-1.001145
4,26,1.0,1,3.806662,3.0,0.693147,0,0,1,0,0,0,0,0,0,0,0,0,1,0.718939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54803,14,1.0,1,3.871201,3.0,2.833213,0,0,1,0,1,0,0,0,0,0,0,0,1,1.092871
54804,27,2.0,1,3.610918,2.0,1.791759,0,0,0,0,0,0,0,0,1,0,0,0,0,-0.552428
54805,1,1.0,1,3.295837,5.0,1.098612,1,0,1,0,0,0,0,0,0,0,0,0,0,1.167657
54806,9,1.0,1,3.367296,1.0,0.693147,0,0,1,0,1,0,0,0,0,0,0,1,0,-1.375077


In [9]:
y.unique()

array([0, 1], dtype=int64)